In [ ]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [11]:
from src.eunms import Model_Type, Scheduler_Type
from src.utils.enums_utils import get_pipes
from src.config import RunConfig

from main import run as invert
from main import create_noise_list
from diffusers.utils import make_image_grid
import torch

# from ptp_utils import AttentionStore
# import ptp_utils
from attention_maps_utils_by_timesteps import (
    # from attention_maps_utils import (
    attn_maps,
    cross_attn_init,
    register_cross_attention_hook,
    set_layer_with_name_and_path,
    preprocess,
    visualize_and_save_attn_map,
)


from PIL import Image

In [12]:
device = "cuda"
model_type = Model_Type.SDXL_Turbo
scheduler_type = Scheduler_Type.EULER
# model_type = Model_Type.SD21model_type = Model_Type.SDXL
# model_type = Model_Type.SDXL
# scheduler_type = Scheduler_Type.DDIM
pipe_inversion, pipe_inference = get_pipes(model_type, scheduler_type, device=device)

Keyword arguments {'safety_checker': None} are not expected by StableDiffusionXLImg2ImgPipeline and will be ignored.
Loading pipeline components...: 100%|███████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 10.46it/s]


In [13]:
inputs_images = [
    (
        Image.open("/home/lab/yairshp/projects/insert_object/benchmark/bed.jpeg")
        .convert("RGB")
        .resize((512, 512))
    ),
    Image.open("/home/lab/yairshp/projects/insert_object/benchmark/desk.jpeg")
    .convert("RGB")
    .resize((512, 512)),
    Image.open("/home/lab/yairshp/projects/insert_object/benchmark/cabinet.jpeg")
    .convert("RGB")
    .resize((512, 512)),
    Image.open("/home/lab/yairshp/projects/insert_object/benchmark/face.jpg")
    .convert("RGB")
    .resize((512, 512)),
    # Image.open("example_images/lion.jpeg").convert("RGB").resize((512, 512)),
]
prompts = [
    "a bed",
    # "a desk",
    "a computer monitor",
    # "a cabinet",
    "a dresser",
    "a man",
    # "a man wearing a grey jacket",
    # "a photo",
    # "a photo",
    # "a photo",
    # "a photo",
]
edit_propmts = [
    "a bed and a pillow",
    # "a desk and a plant",
    "a computer monitor and a plant",
    # "a cabinet and a vase",
    "a dresser and a vase",
    "a man and a hat",
    # "a man wearing a grey jacket and a hat",
    # "a photo and a pillow",
    # "a photo and a plant",
    # "a photo and a vase",
    # "a photo and a hat",
]

# input_image = Image.open("example_images/lion.jpeg").convert("RGB").resize((512, 512))
# prompt = "a lion"
# edit_propmt = "a lion"

config = RunConfig(
    model_type=model_type,
    scheduler_type=scheduler_type,
    # guidance_scale=1.0,
    noise_regularization_lambda_kl=0.05,
    noise_regularization_lambda_ac=10,
    # inversion_max_step=1,
    # num_inversion_steps=50,
    # num_inference_steps=50,
    # num_renoise_steps=1,
    # perform_noise_correction=False,
)

In [ ]:
inv_latents = []
noises = []
for i, (input_image, prompt) in enumerate(zip(inputs_images, prompts)):
    edit_img, inv_latent, noise, all_latents = invert(
        input_image,
        prompt,
        config,
        pipe_inversion=pipe_inversion,
        pipe_inference=pipe_inference,
        do_reconstruction=False,
    )
    inv_latents.append(inv_latent)
    noises.append(noise)

In [ ]:
# cross_attn_init()
# pipe_inference.unet = set_layer_with_name_and_path(pipe_inference.unet)
# pipe_inference.unet = register_cross_attention_hook(pipe_inference.unet)

In [ ]:
img_idx = 0
edit_images = []
for i, (inv_latent, noise, edit_prompt) in enumerate(
    zip(inv_latents, noises, edit_propmts)
):
    # if not i == img_idx:
    #     continue
    # generator = torch.Generator().manual_seed(config.seed)
    pipe_inference.scheduler.set_noise_list(noise)
    pipe_inference.cfg = config
    img = pipe_inference(
        prompt=edit_prompt,
        num_inference_steps=config.num_inference_steps,
        negative_prompt=prompt,
        image=inv_latent,
        strength=config.inversion_max_step,
        denoising_start=1.0 - config.inversion_max_step,
        guidance_scale=config.guidance_scale,
    ).images[0]
    edit_images.append(img)

In [ ]:
# for input_image, edit_img in zip(inputs_images, edit_images):
# i = 0
img_idx = 0
make_image_grid([inputs_images[img_idx], edit_images[img_idx]], 1, 2)
# make_image_grid([inputs_images[img_idx], edit_images[0]], 1, 2)

In [ ]:
# # attn_map = preprocess(max_height=512, max_width=512)
# visualize_and_save_attn_map(attn_map, pipe_inference.tokenizer, edit_propmts[3])
from attention_maps_utils_by_timesteps import get_attn_maps

attn_maps = get_attn_maps()

In [ ]:
timestep = 3
attn_map = preprocess(attn_maps[timestep], 512, 512)
attn_map_img = visualize_and_save_attn_map(
    attn_map,
    pipe_inference.tokenizer,
    edit_propmts[img_idx],
    edit_propmts[img_idx].split()[-1],
)

In [ ]:
attn_map_img

In [ ]:
import numpy as np

attn_map_arr = np.array(attn_map_img)

In [ ]:
max_index = np.argmax(attn_map_arr)
max_x = max_index % attn_map_arr.shape[1]
max_y = max_index // attn_map_arr.shape[1]
print("Max Value Coordinates: x =", max_x, ", y =", max_y)

In [ ]:
import torch
from PIL import Image
from transformers import SamModel, SamProcessor

device = "cuda" if torch.cuda.is_available() else "cpu"
model = SamModel.from_pretrained("facebook/sam-vit-huge").to(device)
processor = SamProcessor.from_pretrained("facebook/sam-vit-huge")

# img_url = "https://huggingface.co/ybelkada/segment-anything/resolve/main/assets/car.png"
# raw_image = Image.open(requests.get(img_url, stream=True).raw).convert("RGB")
input_points = [[[max_x, max_y]]]  # 2D location of a window in the image

inputs = processor(edit_images, input_points=input_points, return_tensors="pt").to(
    device
)
with torch.no_grad():
    outputs = model(**inputs)

masks = processor.image_processor.post_process_masks(
    outputs.pred_masks.cpu(),
    inputs["original_sizes"].cpu(),
    inputs["reshaped_input_sizes"].cpu(),
)
scores = outputs.iou_scores

In [ ]:
masks = masks[0].squeeze().numpy()

In [ ]:
masks.shape

In [ ]:
from PIL import Image

mask_images = []
for mask in masks:
    mask_image = Image.fromarray(mask.astype("uint8") * 255)
    mask_images.append(mask_image)

In [ ]:
mask_images[0]

In [ ]:
import numpy as np
from PIL import Image

# Convert the image and mask to numpy arrays
image_np = np.array(inputs_images[img_idx])
mask_np = np.array(mask_images[0])

# Create a copy of the image
masked_image_np = np.copy(image_np)

# Set the pixels inside the mask to red
masked_image_np[mask_np == 255] = [255, 0, 0]  # Set RGB values to [255, 0, 0] for red

# Create a PIL Image from the numpy array
masked_image = Image.fromarray(masked_image_np)

# Display the masked image
masked_image

In [ ]:
from PIL import Image
import requests
from transformers import Blip2Processor, Blip2ForConditionalGeneration
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
model = Blip2ForConditionalGeneration.from_pretrained(
    "Salesforce/blip2-opt-2.7b", torch_dtype=torch.float16
).to(
    device
)  # doctest: +IGNORE_RESULT

In [ ]:
images = [
    (
        Image.open("/home/lab/yairshp/projects/insert_object/benchmark/bed.jpeg")
        .convert("RGB")
        .resize((512, 512))
    ),
    Image.open("/home/lab/yairshp/projects/insert_object/benchmark/desk.jpeg")
    .convert("RGB")
    .resize((512, 512)),
    Image.open("/home/lab/yairshp/projects/insert_object/benchmark/cabinet.jpeg")
    .convert("RGB")
    .resize((512, 512)),
    Image.open("/home/lab/yairshp/projects/insert_object/benchmark/face.jpg")
    .convert("RGB")
    .resize((512, 512)),
]

object_images = [
    Image.open("/home/lab/yairshp/projects/insert_object/benchmark/objects/hat/hat.png")
    .convert("RGB")
    .resize((512, 512)),
    Image.open(
        "/home/lab/yairshp/projects/insert_object/benchmark/objects/vase/vase.jpeg"
    )
    .convert("RGB")
    .resize((512, 512)),
]

In [ ]:
sentences = []
# for img in object_images:
# for img in images:
prompt = "Question: what object is in this image (in one word)? Answer:"
inputs = processor(
    images=object_images,
    text=[prompt for i in range(len(object_images))],
    return_tensors="pt",
).to(
    # inputs = processor(images=img, text=prompt, return_tensors="pt").to(
    device="cuda",
    dtype=torch.float16,
)

generated_ids = model.generate(**inputs)
generated_texts = processor.batch_decode(generated_ids, skip_special_tokens=True)
for generated_text in generated_texts:
    # strip generated text from both spaces and end of line characters
    generated_text = generated_text.strip().replace("\n", "")
    print(generated_text)
    sentences.append(generated_text)

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")

for sentence in sentences:
    doc = nlp(sentence)
    sents = list(doc.sents)
    nouns = []
    for token in doc:
        if token.pos_ != "NOUN":
            continue
        # if token.dep_ == "compound":
        nouns.append(token.text)
    print(" ".join(nouns))

In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
import torch

from diffusers import StableDiffusionXLPipeline

# from stable_diffusion_xl_return_latents import StableDiffusionXLPipeline
from diffusers import AutoPipelineForText2Image

pipe = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    # torch_dtype=torch.float16,
    # "stabilityai/sdxl-turbo",
    torch_dtype=torch.float16,
)
# pipe = AutoPipelineForText2Image.from_pretrained(
#     "stabilityai/sdxl-turbo",
#     torch_dtype=torch.float16,
#     variant="fp16",
# )
pipe.to("cuda")

/cortex/users/yairshp/miniconda3/envs/renoise_inversion/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/cortex/users/yairshp/miniconda3/envs/renoise_inversion/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/cortex/users/yairshp/miniconda3/envs/renoise_inversion/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/cortex/users/yairshp/miniconda3/envs/renoise_inversion/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarnin

StableDiffusionXLPipeline {
  "_class_name": "StableDiffusionXLPipeline",
  "_diffusers_version": "0.24.0",
  "_name_or_path": "stabilityai/stable-diffusion-xl-base-1.0",
  "feature_extractor": [
    null,
    null
  ],
  "force_zeros_for_empty_prompt": true,
  "image_encoder": [
    null,
    null
  ],
  "scheduler": [
    "diffusers",
    "EulerDiscreteScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "text_encoder_2": [
    "transformers",
    "CLIPTextModelWithProjection"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "tokenizer_2": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [6]:
# prompts = [
#     # "A hotel bed ready for use",
#     # "A skier quickly going down an empty hill",
#     # "a bench in a park",
#     # "a kitchen with a stove and a sink",
#     "A line of snowboarders in the woods",
#     # "A bike leaning against a brown rack with a tree in the background",
#     # "A stop sign in a dirt road",
#     # "A small and clean bathroom",
#     # "A laptop and some books on a desk",
#     # "A cow graze on a mountain overlooking a city",
#     # "A marina with white sailboats in front of a small town of buildings and homes that border a forest behind them",
#     # "A person jumps in a field of wheat trying to reach a fribee hurtling above him",
#     # "",
#     # "",
# ]
prompts = [
    "A skier quickly going down an empty hill",
    "A stop sign in a dirt road",
    # "a bench in a park",
    "A red vintage car parked on a cliff overlooking a vast canyon landscape",
    "A picnic basket and a blanket laid out on a grassy hillside",
    "A kite soaring high against the backdrop of a cloudy sky, with a child running across an open field below",
    "A camping tent pitched in a lush forest clearing, with towering redwood trees surrounding the site",
    "A hammock suspended between two palm trees on a white sandy beach, with crystal-clear turquoise waters in the background",
    "A parasol in the beach",
]

# output, init_latents = pipe(prompt, num_inference_steps=4)
for prompt in prompts:
    image = pipe(
        prompt=f"{prompt}",
        # num_inference_steps=2,
        # guidance_scale=7.5,
        # guidance_scale=0.0,
    ).images[0]
    image.save(f"example_images/generated_2/{prompt}.png")

  0%|          | 0/50 [00:00<?, ?it/s]

100%|██████████| 50/50 [00:12<00:00,  3.88it/s]
